In [1]:
import itertools
import numpy as np
import pandas as pd
from math import trunc,radians,sin,cos,sqrt,atan2
from datetime import datetime
import time
import folium
from folium import plugins

############## Inputs#######################

df = pd.read_excel('C:/Users/Gabri/OneDrive/Documentos/Martins/sub_set_df.xlsx') #database deve estar filtrado por data do mais ao antigo ao mais novo
df_armazens = [[-12.9706, -38.5126]]
df_clientes = df[['VLRLTTCLI_VND','VLRLNTCLI_VND']].to_numpy().copy()
df_datas = df[['DATA_PED']].copy()
df_filiais_b = pd.read_excel('C:/Users/Gabri/OneDrive/Documentos/Martins/TABELA_FILIAIS_11_ATUAIS.xlsx')
df_filiais = df_filiais_b[['VLRLTTMNCCANFIL', 'VLRLNTMNCCANFIL']].to_numpy().copy()
nome_coordenada = {}
indice_nome = {}
#print(df_filiais_b['NOMCID'][0])


n = 11 #numero de filiais por combinacao



#associaçao entre nome e coordenada
def totuple(array):
    try:
        return tuple(totuple(i) for i in array)
    except TypeError:
        return array

hash_array = totuple(df_filiais)
print(hash_array)
for indice in range(len(df_filiais)):
    indice_nome[hash_array[indice]] = indice
    nome_coordenada[df_filiais_b['NOMCID'][indice]] = df_filiais[indice]
print(indice_nome)
#print(nome_coordenada)
#fim da associaçao
combinacoes = list(itertools.combinations(df_filiais, n))
#print(combinacoes)
combinacoes = np.array(combinacoes)

dicionario_combinacoes = {}
for idx, combinacao in enumerate(combinacoes):
    dicionario_combinacoes[idx + 1] = combinacao

print(dicionario_combinacoes)
len(dicionario_combinacoes)


((-12.9706, -38.5126), (-14.8496, -40.8393), (-12.698, -38.324), (-14.4716, -39.1642), (-12.153, -44.99), (-16.378, -39.58), (-9.4048, -38.238), (-14.2123, -42.7676), (-11.2999, -41.8568), (-10.4658, -40.1808), (-12.528, -40.307))
{(-12.9706, -38.5126): 0, (-14.8496, -40.8393): 1, (-12.698, -38.324): 2, (-14.4716, -39.1642): 3, (-12.153, -44.99): 4, (-16.378, -39.58): 5, (-9.4048, -38.238): 6, (-14.2123, -42.7676): 7, (-11.2999, -41.8568): 8, (-10.4658, -40.1808): 9, (-12.528, -40.307): 10}
{1: array([[-12.9706, -38.5126],
       [-14.8496, -40.8393],
       [-12.698 , -38.324 ],
       [-14.4716, -39.1642],
       [-12.153 , -44.99  ],
       [-16.378 , -39.58  ],
       [ -9.4048, -38.238 ],
       [-14.2123, -42.7676],
       [-11.2999, -41.8568],
       [-10.4658, -40.1808],
       [-12.528 , -40.307 ]])}


1

In [2]:

############## Calculo de distancias####################
def distancia_lat_long_km(filial,pedido): # algoritmo de calculo da distancia feito pela Myllene
    # converte graus em radianos
    lat1, lon1, lat2, lon2 = map(radians, [filial[0], filial[1], pedido[0], pedido[1]])
    
    # raio da Terra em km
    R = 6371
    
    # diferença de latitude e longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # fórmula Haversine
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))
    distancia = R*c
    
    return distancia






In [3]:
print(len(df_filiais))

11


In [4]:
########## Função generica de calculo da distancia total do dia #########

def calcular(filiais,clientes):
    atribuicao = {}
    #print(f"os clientes sao: {clientes}")
    #print()
    for item in df_filiais:
        hash_item = totuple(item)
        atribuicao[hash_item] = []
    somatorio = []
    #print(atribuicao)
    for pedido in clientes :
        distancia_min = 100000  # distancia iniciada em um valor alto, para garantir a troca no primeiro loop
        i = -1 #variavel gambiarra pra achar o index do array numpy
        for filial in filiais:          
            dist = distancia_lat_long_km(filial,pedido)
            #dist += distancia_lat_long_km(df_armazens[0],filial) # acrescenta a distancia da filial ao armazem
            i += 1
            if(dist < distancia_min): # troca somente se a distancia total for menor que a anterior
                distancia_min = dist
                hash_fil = totuple(filial)
            #print(f"O pedido {pedido} escolheu a filial =={salva_filial}")
        atribuicao[hash_fil].append(pedido)
        somatorio.append(distancia_min)
    houve_pedido = {}
    for i in filiais:
      hi = totuple(i)
      if(atribuicao[hi] != 0):
            houve_pedido[hi]= True
    #print(houve_pedido)
    #print(f"O valor antes dos armazens é: {sum(somatorio)}")
    for filial, escolha in houve_pedido.items(): 
        if(escolha):
            somatorio.append(distancia_lat_long_km(df_armazens[0],filial))
            
    resultado = sum(somatorio)
    return atribuicao,resultado

In [5]:
if __name__ == '__main__':    
    inicio = time.time()
    Best = 0
    Var = {}
    Plot = {}
    for chave,_ in dicionario_combinacoes.items():
        Plot[chave] = []
    contagem_combinacoes = 0
    for chave, combinacao in dicionario_combinacoes.items():
        #print(f'Combinação {chave}: {combinacao}')
        filiais_df = combinacao
        #print(df_armazens)
        #print(df_clientes)
        ###########definições da funçao de 92 dias
        final = []
        #print(final)
        j = 0
##############################função 92 dias        
        for dia in range(92):
            Clientes_dia = []
            for data in df_datas.DATA_PED[j:]:
                Clientes_dia.append(df_clientes[j])
                if(j == len(df_clientes)-1):
                    j += 1
                    break
                if(data != df_datas.DATA_PED[j+1]):
                    j += 1
                    break
                j += 1
            #print(f"Dia {dia} = {len(Clientes_dia)} pedidos")
            pedido_filial , soma = calcular(filiais_df,Clientes_dia)
            Plot[chave].append(pedido_filial)
            final.append(trunc(soma))
        
################################fim da função
###############condição da funçao de combinações
        #if(contagem_combinacoes == len(combinacao)-1):
           # contagem_combinacoes += 1
           # break
        contagem_combinacoes +=1
##########################################fim das condições###
        #print()
        #print('O Vetor de distâncias é', final)
        #print()
        #print(len(final), 'dias')
#####################calculos de metricas
        #Plot[chave].append( pedido_filial)
        #print(Plot[chave][0])
        res = np.array(final)
        fit = np.sqrt(np.mean((res - np.mean(res))**2))
        #print(f"SMSE = {fit:.1f} km")
        #print()
        Var[chave] = trunc(fit)
        km_rodado = sum(final)
        #print("Total de KM rodado foi de:", km_rodado)
        #print()
    best = min(Var.values())
    print('A quantidade de combinações é', contagem_combinacoes)
    print()
    print(f'Essas foram escolhidas : {n} a {n} de um total de {len(df_filiais)}')
    combinacao_escolhida = list(Var.keys())[list(Var.values()).index(best)]
    print()
    print(f"A melhor combinação foi SMSE = {best} na combinação número " , combinacao_escolhida)
    filiais_escolhidas = []
    _pedidos = Plot[combinacao_escolhida]
    pedidos_por_filial = {}
    #print(_pedidos[0])
    for l in indice_nome.values():
        pedidos_por_filial[l]= []
    indice_escolhidos= []
    #print(pedidos_por_filial)
    #print(_pedidos[10][9])
    
    for filial in dicionario_combinacoes[combinacao_escolhida]:
        hash_filial = totuple(filial)
        idc = indice_nome[hash_filial]
        indice_escolhidos.append(idc)
        for dia in range(92):
            pedidos_por_filial[idc].append(len(_pedidos[dia][hash_filial])) 
        nome_da_filial = list(nome_coordenada.keys())[idc]
        #filiais_escolhidas[idc] = nome_da_filial
        filiais_escolhidas.append(nome_da_filial)
    print()
    print(f"As filiais escolhidas foram :", filiais_escolhidas)
    print()
    contador = 0
    for i in indice_escolhidos :
        print(f"A filial {filiais_escolhidas[contador]} entregou : {sum(pedidos_por_filial[i])} pedidos ao todo \n")
        contador += 1
        if contador >= len(filiais_escolhidas):
            break
###### tratamento dos pedidos filiais
####fim do tratamento pedidos filiais
####################fim dos calculos de metricas
    fim = time.time()
    print()
    print("Tempo de execução: %s"%(fim - inicio))

A quantidade de combinações é 1

Essas foram escolhidas : 11 a 11 de um total de 11

A melhor combinação foi SMSE = 48909 na combinação número  1

As filiais escolhidas foram : ['SALVADOR', 'VITOR DA CONQUISTA', 'CAMACARI', 'ITABUNA', 'BARREIRAS', 'EUNAPOLIS', 'PAULO AFONSO', 'GUANAMBI', 'IRECE', 'SENHOR DO BONFIM', 'ITABERABA']

A filial SALVADOR entregou : 17180 pedidos ao todo 

A filial VITOR DA CONQUISTA entregou : 7575 pedidos ao todo 

A filial CAMACARI entregou : 13167 pedidos ao todo 

A filial ITABUNA entregou : 8206 pedidos ao todo 

A filial BARREIRAS entregou : 7419 pedidos ao todo 

A filial EUNAPOLIS entregou : 8554 pedidos ao todo 

A filial PAULO AFONSO entregou : 5500 pedidos ao todo 

A filial GUANAMBI entregou : 9197 pedidos ao todo 

A filial IRECE entregou : 12907 pedidos ao todo 

A filial SENHOR DO BONFIM entregou : 12348 pedidos ao todo 

A filial ITABERABA entregou : 10406 pedidos ao todo 


Tempo de execução: 5.522187232971191


In [6]:
filiais_escolhidas = dicionario_combinacoes[combinacao_escolhida]
print(filiais_escolhidas)

[[-12.9706 -38.5126]
 [-14.8496 -40.8393]
 [-12.698  -38.324 ]
 [-14.4716 -39.1642]
 [-12.153  -44.99  ]
 [-16.378  -39.58  ]
 [ -9.4048 -38.238 ]
 [-14.2123 -42.7676]
 [-11.2999 -41.8568]
 [-10.4658 -40.1808]
 [-12.528  -40.307 ]]


In [75]:
def calcularP(filiais,clientes):
    Distancia_arm_fil = []
    DISTANCIA_ARM_FILIAL = []
    for pedido in clientes :
        distancia_min = 100000  # distancia iniciada em um valor alto, para garantir a troca no primeiro loop
        i = -1 #variavel gambiarra pra achar o index do array numpy
        for filial in filiais:          
            dist = distancia_lat_long_km(filial,pedido)
            dist_arm = distancia_lat_long_km(df_armazens[0],filial) # acrescenta a distancia da filial ao armazem
            i += 1
            if(dist < distancia_min): # troca somente se a distancia total for menor que a anterior
                distancia_min = dist
                distancia_min_arm = dist_arm
                hash_fil = totuple(filial)
            #print(f"O pedido {pedido} escolheu a filial =={salva_filial}")
        Distancia_arm_fil.append(distancia_min)
        DISTANCIA_ARM_FILIAL.append(distancia_min_arm)
    return Distancia_arm_fil, DISTANCIA_ARM_FILIAL

In [76]:

Pedidos_dia = {}
Pedidos_dia["DISTANCIA_FILIAL_CLIENTE"] = []
Pedidos_dia["DISTANCIA_ARM_FILIAL"] = []
#Pedidos_dia.append("DISTANCIA_FILIAL_CLIENTE")
j = 0
for dia in range(92):
    Clientes_dia = []
    for data in df_datas.DATA_PED[j:]:
        Clientes_dia.append(df_clientes[j])
        if(j == len(df_clientes)-1):
            j += 1
            break
        if(data != df_datas.DATA_PED[j+1]):
            j += 1
            break
        j += 1
    #print(f"Dia {dia} = {len(Clientes_dia)} pedidos")
    pedido_filial, arm_filial = calcularP(filiais_escolhidas,Clientes_dia)
    Pedidos_dia["DISTANCIA_FILIAL_CLIENTE"].extend(pedido_filial)
    Pedidos_dia["DISTANCIA_ARM_FILIAL"].extend(arm_filial)

In [80]:
#print(Pedidos_dia)
print(len(Pedidos_dia))
df_Pedidos = pd.DataFrame(Pedidos_dia["DISTANCIA_FILIAL_CLIENTE"],columns= ["DISTANCIA_FILIAL_CLIENTE"] )
df_arm_fil = pd.DataFrame(Pedidos_dia["DISTANCIA_ARM_FILIAL"], columns=["DISTANCIA_ARM_FILIAL"])
df_Pedidos.head()

2


,DISTANCIA_FILIAL_CLIENTE
0,189.020783
1,189.020783
2,189.020783
3,185.557340
4,169.793655


In [81]:
df_arm_fil.head()

,DISTANCIA_ARM_FILIAL
0,395.894877
1,395.894877
2,395.894877
3,395.894877
4,395.894877


In [43]:
Path_df = "C:/Users/gabri/OneDrive/Documentos/Martins/dfs/SUBSET_DF_FINAL.xlsx"

In [45]:
df_custo = pd.read_excel(Path_df)

In [82]:
print(len(df_custo[["DISTANCIA_FILIAL_CLIENTE"]]))
df_custo[["DISTANCIA_FILIAL_CLIENTE"]] = df_Pedidos
df_custo[["DISTANCIA_ARM_FILIAL"]] = df_arm_fil

112459


In [83]:
df_custo.to_excel("df_custo.xlsx")